# Migration script for Elasticsearch

This script adds metadata to elasticsearch.

In [ ]:
import os

import psycopg2
from elasticsearch import Elasticsearch

from scraper_place.config import CONFIG_DATABASE, STATE_CONTENT_INDEXATION_OK

In [ ]:
es_client = Elasticsearch()

In [ ]:
# Open connection
connection = psycopg2.connect(
    dbname=CONFIG_DATABASE['name'],
    user=CONFIG_DATABASE['username'],
    password=CONFIG_DATABASE['password'],
)
cursor = connection.cursor()

In [ ]:
cursor.execute(
    """
    SELECT
        annonce_id, org_acronym, links_boamp, reference, intitule, objet, reglement_ref,
        filename_reglement, filename_complement, filename_avis, filename_dce,
        fetch_datetime,
        file_size_reglement, file_size_complement, file_size_avis, file_size_dce,
        glacier_id_reglement, glacier_id_complement, glacier_id_avis, glacier_id_dce,
        embedded_filenames_reglement, embedded_filenames_complement, embedded_filenames_avis, embedded_filenames_dce,
        state
    FROM dce
    WHERE state = %s
    ;""",
    (STATE_CONTENT_INDEXATION_OK,),
)
dce_data_list = cursor.fetchall()

In [ ]:
len(dce_data_list)

In [ ]:
for i, dce_data in enumerate(dce_data_list[4746:]):
    (
        annonce_id, org_acronym, links_boamp, reference, intitule, objet, reglement_ref,
        filename_reglement, filename_complement, filename_avis, filename_dce,
        fetch_datetime,
        file_size_reglement, file_size_complement, file_size_avis, file_size_dce,
        glacier_id_reglement, glacier_id_complement, glacier_id_avis, glacier_id_dce,
        embedded_filenames_reglement, embedded_filenames_complement, embedded_filenames_avis, embedded_filenames_dce,
        state
    ) = dce_data

    response = es_client.update(
        index='dce',
        doc_type='dce',
        id='{}-{}'.format(annonce_id, org_acronym),
        body={
            'doc': {
                'annonce_id': annonce_id,
                'org_acronym': org_acronym,
                'links_boamp': links_boamp,
                'reference': reference,
                'intitule': intitule,
                'objet': objet,
                'reglement_ref': reglement_ref,
                'filename_reglement': filename_reglement,
                'filename_complement': filename_complement,
                'filename_avis': filename_avis,
                'filename_dce': filename_dce,
                'fetch_datetime': fetch_datetime,
                'file_size_reglement': file_size_reglement,
                'file_size_complement': file_size_complement,
                'file_size_avis': file_size_avis,
                'file_size_dce': file_size_dce,
                'glacier_id_reglement': glacier_id_reglement,
                'glacier_id_complement': glacier_id_complement,
                'glacier_id_avis': glacier_id_avis,
                'glacier_id_dce': glacier_id_dce,
                'embedded_filenames_reglement': embedded_filenames_reglement,
                'embedded_filenames_complement': embedded_filenames_complement,
                'embedded_filenames_avis': embedded_filenames_avis,
                'embedded_filenames_dce': embedded_filenames_dce,
                'state': state,
            },
        },
    )
    
    if i%100 == 0:
        print('.')


In [ ]:
es_client.get(
        index='dce',
        doc_type='dce',
        id='350004-g7h',
)

In [ ]:
cursor.close()
connection.close()